# Работа с Core в SQL Alchemy 

<a target="_blank" href="https://colab.research.google.com/github/sozykin/middle_python/blob/main/06/06_core.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [29]:
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import create_engine
from sqlalchemy import URL
from sqlalchemy import select, insert, update, delete

In [8]:
url_object = URL.create(
    "postgresql+psycopg2",
    username="postgres",
    password="1234qwer",  
    host="localhost",
    database="postgres",
)

In [9]:
engine = create_engine(url_object, echo=True)

In [18]:
metadata_obj = MetaData()

In [19]:
names_table = Table(
        "names_core",
         metadata_obj,
         Column("id", Integer),
        Column("name", String),
        Column('number_of_persons', Integer),
        Column('year', Integer),
        Column('month', String)
)

In [20]:
metadata_obj.create_all(engine)

2023-06-19 08:18:18,349 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 08:18:18,350 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2023-06-19 08:18:18,350 INFO sqlalchemy.engine.Engine [cached since 358.2s ago] {'table_name': 'names_core', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2023-06-19 08:18:18,352 INFO sqlalchemy.engine.Engine 
CREATE TABLE names_core (
	id INTEGER, 
	name VARCHAR, 
	number_of_persons INTEGER, 
	year INTEGER, 
	month VARCHAR
)


2023-06-19 08:18:18,352 INFO sqlalchemy.engine.Engin

## Вставка данных

In [21]:
stmt = insert(names_table).values(id=1, name='Мария', number_of_persons=252, year=2015, month='январь')

In [22]:
print(stmt)

INSERT INTO names_core (id, name, number_of_persons, year, month) VALUES (:id, :name, :number_of_persons, :year, :month)


In [23]:
with engine.connect() as conn:
    conn.execute(stmt)
    conn.commit()

2023-06-19 08:18:26,603 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 08:18:26,604 INFO sqlalchemy.engine.Engine INSERT INTO names_core (id, name, number_of_persons, year, month) VALUES (%(id)s, %(name)s, %(number_of_persons)s, %(year)s, %(month)s)
2023-06-19 08:18:26,604 INFO sqlalchemy.engine.Engine [generated in 0.00089s] {'id': 1, 'name': 'Мария', 'number_of_persons': 252, 'year': 2015, 'month': 'январь'}
2023-06-19 08:18:26,605 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
stmt = insert(names_table).values(id=2, name='Анастасия', number_of_persons=224, year=2015, month='январь')

In [25]:
with engine.connect() as conn:
    conn.execute(stmt)
    conn.commit()

2023-06-19 08:19:48,015 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 08:19:48,016 INFO sqlalchemy.engine.Engine INSERT INTO names_core (id, name, number_of_persons, year, month) VALUES (%(id)s, %(name)s, %(number_of_persons)s, %(year)s, %(month)s)
2023-06-19 08:19:48,016 INFO sqlalchemy.engine.Engine [cached since 81.41s ago] {'id': 2, 'name': 'Анастасия', 'number_of_persons': 224, 'year': 2015, 'month': 'январь'}
2023-06-19 08:19:48,017 INFO sqlalchemy.engine.Engine COMMIT


## Извлечение данных

In [26]:
stmt = select(names_table)

In [27]:
print(stmt)

SELECT names_core.id, names_core.name, names_core.number_of_persons, names_core.year, names_core.month 
FROM names_core


In [28]:
with engine.connect() as conn:
    results = conn.execute(stmt)
    for row in results:
        print(row)

2023-06-19 08:21:33,237 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 08:21:33,237 INFO sqlalchemy.engine.Engine SELECT names_core.id, names_core.name, names_core.number_of_persons, names_core.year, names_core.month 
FROM names_core
2023-06-19 08:21:33,237 INFO sqlalchemy.engine.Engine [generated in 0.00071s] {}
(1, 'Мария', 252, 2015, 'январь')
(2, 'Анастасия', 224, 2015, 'январь')
2023-06-19 08:21:33,239 INFO sqlalchemy.engine.Engine ROLLBACK


## Изменение данных

In [30]:
stmt = update(names_table).where(names_table.c.id == 1).values(name='Анна')

In [31]:
print(stmt)

UPDATE names_core SET name=:name WHERE names_core.id = :id_1


In [32]:
with engine.connect() as conn:
    conn.execute(stmt)

2023-06-19 08:34:46,144 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 08:34:46,145 INFO sqlalchemy.engine.Engine UPDATE names_core SET name=%(name)s WHERE names_core.id = %(id_1)s
2023-06-19 08:34:46,145 INFO sqlalchemy.engine.Engine [generated in 0.00116s] {'name': 'Анна', 'id_1': 1}
2023-06-19 08:34:46,156 INFO sqlalchemy.engine.Engine ROLLBACK


In [33]:
with engine.connect() as conn:
    conn.execute(stmt)
    conn.commit()

2023-06-19 08:35:11,786 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 08:35:11,787 INFO sqlalchemy.engine.Engine UPDATE names_core SET name=%(name)s WHERE names_core.id = %(id_1)s
2023-06-19 08:35:11,787 INFO sqlalchemy.engine.Engine [cached since 25.64s ago] {'name': 'Анна', 'id_1': 1}
2023-06-19 08:35:11,788 INFO sqlalchemy.engine.Engine COMMIT


## Удаление данных

In [34]:
stmt = delete(names_table).where(names_table.c.id == 2)

In [35]:
print(stmt)

DELETE FROM names_core WHERE names_core.id = :id_1


In [36]:
with engine.connect() as conn:
    conn.execute(stmt)
    conn.commit()

2023-06-19 08:37:07,714 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 08:37:07,715 INFO sqlalchemy.engine.Engine DELETE FROM names_core WHERE names_core.id = %(id_1)s
2023-06-19 08:37:07,715 INFO sqlalchemy.engine.Engine [generated in 0.00110s] {'id_1': 2}
2023-06-19 08:37:07,716 INFO sqlalchemy.engine.Engine COMMIT
